# AWS Glue Studio Notebook
##### You are now running a AWS Glue Studio notebook; To start using your notebook you need to start an AWS Glue Interactive Session.


#### Optional: Run this cell to see available notebook commands ("magics").


####  Run this cell to set up and start your interactive session.


In [1]:
%idle_timeout 2880
%glue_version 4.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.sql import functions as F
from pyspark.storagelevel import StorageLevel
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
 
glueContext._jsc.hadoopConfiguration().set("fs.s3.useRequesterPaysHeader","true") ## this is needed for permissions
spark._jsc.hadoopConfiguration().set("fs.s3.useRequesterPaysHeader","true") ## this is needed for permissions
 
spark = glueContext.spark_session
spark.catalog.clearCache()
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.7 
Current idle_timeout is None minutes.
idle_timeout has been set to 2880 minutes.
Setting Glue version to: 4.0
Previous worker type: None
Setting new worker type to: G.1X
Previous number of workers: None
Setting new number of workers to: 5
Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 5
Idle Timeout: 2880
Session ID: a1b531ea-a229-4170-b6e5-f0cb10994ea8
Applying the following default arguments:
--glue_kernel_version 1.0.7
--enable-glue-datacatalog true
Waiting for session a1b531ea-a229-4170-b6e5-f0cb10994ea8 to get into ready status...
Session a1b531ea-a229-4170-b6e5-f0cb10994ea8 ha

In [6]:
test_query = """
SELECT 
    primary_intent_detail,  
    CAST(contact_dt AS DATE) AS contact_dt, 
    COUNT(DISTINCT sub_contact_id) AS sub_contact_id, 
    COUNT(DISTINCT CASE WHEN selfservice_containment = 1 THEN sub_contact_id END) AS contained, 
    CASE 
        WHEN COUNT(DISTINCT sub_contact_id) > 0 THEN
            ROUND(CAST(SUM(CASE WHEN selfservice_containment = 1 THEN 1 ELSE 0 END) AS DOUBLE) 
            / COUNT(DISTINCT sub_contact_id) * 100, 2)
        ELSE
            0
    END AS containment_rate
FROM 
    ota_data_assets_temp.omni_intent_cntct_fact 
WHERE 
    CAST(contact_dt AS DATE) BETWEEN date_add(DATE '2024-08-27', -90) AND DATE '2024-08-27' 
    AND primary_intent = 'Equipment Support'
    AND initial_channel = 'CoxApp'
    AND lob = 'R'
    AND primary_intent_detail IN ('PnP', 'SmartHelp')
GROUP BY 
    primary_intent_detail, contact_dt
ORDER BY 
    contact_dt DESC
"""

In [7]:
df = spark.sql(test_query)
df.show()

+---------------------+----------+--------------+---------+----------------+
|primary_intent_detail|contact_dt|sub_contact_id|contained|containment_rate|
+---------------------+----------+--------------+---------+----------------+
|                  PnP|2024-08-27|          1839|     1593|           86.62|
|            SmartHelp|2024-08-27|          1453|     1346|           92.64|
|            SmartHelp|2024-08-26|          1500|     1399|           93.27|
|                  PnP|2024-08-26|          1932|     1658|           85.82|
|            SmartHelp|2024-08-25|          1208|     1143|           94.62|
|                  PnP|2024-08-25|          1900|     1680|           88.42|
|                  PnP|2024-08-24|          1983|     1727|           87.09|
|            SmartHelp|2024-08-24|          1436|     1347|            93.8|
|            SmartHelp|2024-08-23|          1694|     1606|           94.81|
|                  PnP|2024-08-23|          2351|     2053|           87.32|

In [8]:
df.printSchema()

root
 |-- primary_intent_detail: string (nullable = true)
 |-- contact_dt: date (nullable = true)
 |-- sub_contact_id: long (nullable = false)
 |-- contained: long (nullable = false)
 |-- containment_rate: double (nullable = true)


In [9]:
df.count()

182


In [ ]:
first_table = "s3://cci-dig-aicoe-data-sb/processed/Health_Score/first_table/first_table.csv"

In [3]:
first_df = spark.read.csv(first_table, header=True, inferSchema=True)
first_df.show(5)

+---------+--------------------+--------------------+---------------------+------------+-----------------+-------------+--------------------+--------------+--------------------+-------+-------+------------------+----------------+-------------------+------------+--------+----------+---------------+---------+-------------------+
|metric_id|             metrics|       display_names|operating_system_type|feature_name|unique_identifier|metric_nature|             level_1|       level_2|             level_3|level_4|level_5|      hierarchy_id|   parent_id_old|metric_sequence_num|metric_seqno|level_no|feature_id|feature_seq_num|parent_id|          create_dt|
+---------+--------------------+--------------------+---------------------+------------+-----------------+-------------+--------------------+--------------+--------------------+-------+-------+------------------+----------------+-------------------+------------+--------+----------+---------------+---------+-------------------+
|        1|DR

In [4]:
first_df.printSchema()

root
 |-- metric_id: integer (nullable = true)
 |-- metrics: string (nullable = true)
 |-- display_names: string (nullable = true)
 |-- operating_system_type: string (nullable = true)
 |-- feature_name: string (nullable = true)
 |-- unique_identifier: string (nullable = true)
 |-- metric_nature: string (nullable = true)
 |-- level_1: string (nullable = true)
 |-- level_2: string (nullable = true)
 |-- level_3: string (nullable = true)
 |-- level_4: string (nullable = true)
 |-- level_5: string (nullable = true)
 |-- hierarchy_id: string (nullable = true)
 |-- parent_id_old: string (nullable = true)
 |-- metric_sequence_num: integer (nullable = true)
 |-- metric_seqno: integer (nullable = true)
 |-- level_no: double (nullable = true)
 |-- feature_id: integer (nullable = true)
 |-- feature_seq_num: integer (nullable = true)
 |-- parent_id: double (nullable = true)
 |-- create_dt: timestamp (nullable = true)


In [5]:
from pyspark.sql import functions as F

# Assuming `df` is your DataFrame
first_df_transformed = first_df.withColumn("level_no", first_df["level_no"].cast("integer")) \
                   .withColumn("parent_id", first_df["parent_id"].cast("integer")) \
                   .withColumn("create_dt", F.to_date(first_df["create_dt"], "yyyy-MM-dd HH:mm"))

# Show the transformed DataFrame
first_df_transformed.show()


+---------+--------------------+--------------------+---------------------+------------+-----------------+-------------+--------------------+--------------------+--------------------+-------+-------+------------------+----------------+-------------------+------------+--------+----------+---------------+---------+----------+
|metric_id|             metrics|       display_names|operating_system_type|feature_name|unique_identifier|metric_nature|             level_1|             level_2|             level_3|level_4|level_5|      hierarchy_id|   parent_id_old|metric_sequence_num|metric_seqno|level_no|feature_id|feature_seq_num|parent_id| create_dt|
+---------+--------------------+--------------------+---------------------+------------+-----------------+-------------+--------------------+--------------------+--------------------+-------+-------+------------------+----------------+-------------------+------------+--------+----------+---------------+---------+----------+
|        1|DR_Visits t

In [6]:
first_df_transformed.printSchema()

root
 |-- metric_id: integer (nullable = true)
 |-- metrics: string (nullable = true)
 |-- display_names: string (nullable = true)
 |-- operating_system_type: string (nullable = true)
 |-- feature_name: string (nullable = true)
 |-- unique_identifier: string (nullable = true)
 |-- metric_nature: string (nullable = true)
 |-- level_1: string (nullable = true)
 |-- level_2: string (nullable = true)
 |-- level_3: string (nullable = true)
 |-- level_4: string (nullable = true)
 |-- level_5: string (nullable = true)
 |-- hierarchy_id: string (nullable = true)
 |-- parent_id_old: string (nullable = true)
 |-- metric_sequence_num: integer (nullable = true)
 |-- metric_seqno: integer (nullable = true)
 |-- level_no: integer (nullable = true)
 |-- feature_id: integer (nullable = true)
 |-- feature_seq_num: integer (nullable = true)
 |-- parent_id: integer (nullable = true)
 |-- create_dt: date (nullable = true)


In [7]:
first_output_path = "s3://cci-dig-aicoe-data-sb/processed/Health_Score/healthscore_master/"
first_write_df = first_df_transformed.write.format("parquet").mode("append").save(first_output_path)

In [8]:
second_table = "s3://cci-dig-aicoe-data-sb/processed/Health_Score/second_table/second_table.csv"

In [9]:
second_df = spark.read.csv(second_table, header=True, inferSchema=True)
second_df.show(5)

+---------+-------------------+----------+-------+
|metric_id|          create_dt|      Date|  Value|
+---------+-------------------+----------+-------+
|        1|2025-02-06 00:00:00|29-08-2024|71742.0|
|        2|2025-02-06 00:00:00|29-08-2024|28280.0|
|        3|2025-02-06 00:00:00|29-08-2024|    1.0|
|        4|2025-02-06 00:00:00|29-08-2024|    1.0|
|        5|2025-02-06 00:00:00|29-08-2024|26212.0|
+---------+-------------------+----------+-------+
only showing top 5 rows


In [10]:
second_df.printSchema()

root
 |-- metric_id: integer (nullable = true)
 |-- create_dt: timestamp (nullable = true)
 |-- Date: string (nullable = true)
 |-- Value: double (nullable = true)


In [11]:
from pyspark.sql import functions as F

# Assuming df is your DataFrame
second_df_transformed = second_df.withColumn("create_dt", F.to_date(second_df["create_dt"])) \
                   .withColumn("Date", F.to_date(second_df["Date"], "dd-MM-yyyy")) \
                   .withColumn("Value", second_df["Value"].cast("integer"))

# Show the transformed DataFrame
second_df_transformed.show()


+---------+----------+----------+-----+
|metric_id| create_dt|      Date|Value|
+---------+----------+----------+-----+
|        1|2025-02-06|2024-08-29|71742|
|        2|2025-02-06|2024-08-29|28280|
|        3|2025-02-06|2024-08-29|    1|
|        4|2025-02-06|2024-08-29|    1|
|        5|2025-02-06|2024-08-29|26212|
|        6|2025-02-06|2024-08-29|25656|
|        7|2025-02-06|2024-08-29|    0|
|        8|2025-02-06|2024-08-29| 3022|
|        9|2025-02-06|2024-08-29| 1969|
|       10|2025-02-06|2024-08-29|  227|
|       11|2025-02-06|2024-08-29| 6387|
|       12|2025-02-06|2024-08-29| 5111|
|       13|2025-02-06|2024-08-29| 2449|
|       14|2025-02-06|2024-08-29| 2675|
|       15|2025-02-06|2024-08-29| 2005|
|       16|2025-02-06|2024-08-29|  784|
|       17|2025-02-06|2024-08-29|  137|
|       18|2025-02-06|2024-08-29|  780|
|       19|2025-02-06|2024-08-29| 2236|
|       20|2025-02-06|2024-08-29| 3630|
+---------+----------+----------+-----+
only showing top 20 rows


In [12]:
second_df_transformed.printSchema()

root
 |-- metric_id: integer (nullable = true)
 |-- create_dt: date (nullable = true)
 |-- Date: date (nullable = true)
 |-- Value: integer (nullable = true)


In [13]:
second_output_path = "s3://cci-dig-aicoe-data-sb/processed/Health_Score/healthscore_actual_data/"
second_table_write_df = second_df_transformed.write.format("parquet").mode("append").save(second_output_path)

In [14]:
third_Table = "s3://cci-dig-aicoe-data-sb/processed/Health_Score/third_table/third_table.csv"

In [15]:
third_df = spark.read.csv(third_Table, header=True, inferSchema=True)
third_df.show(5)

+----------+---------+------------+--------------------+--------------------+---------------------+----------+-------+
| create_dt|metric_id|feature_name|             metrics|       display_names|operating_system_type|      Date|  Value|
+----------+---------+------------+--------------------+--------------------+---------------------+----------+-------+
|06-02-2025|        1|    BIL LING|DR_Visits to Bill...|Billing Section V...|            Apple iOS|29-08-2024|71742.0|
|06-02-2025|        2|    BIL LING|DR_Billing Make a...|       Make Payments|            Apple iOS|29-08-2024| 39.419|
|06-02-2025|        3|    BIL LING|DR_Client Errors ...|Client Errors on ...|            Apple iOS|29-08-2024|  0.004|
|06-02-2025|        4|    BIL LING|DR_Server Errors ...|Server Errors on ...|            Apple iOS|29-08-2024|  0.004|
|06-02-2025|        5|    BIL LING|   DR_Payment Submit|      Payment Submit|            Apple iOS|29-08-2024| 36.536|
+----------+---------+------------+-------------

In [16]:
third_df.printSchema()

root
 |-- create_dt: string (nullable = true)
 |-- metric_id: integer (nullable = true)
 |-- feature_name: string (nullable = true)
 |-- metrics: string (nullable = true)
 |-- display_names: string (nullable = true)
 |-- operating_system_type: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Value: double (nullable = true)


In [17]:
from pyspark.sql import functions as F

# Assuming df is your DataFrame
third_df_transformed = third_df.withColumn("create_dt", F.to_date(third_df["create_dt"], "dd-MM-yyyy")) \
                   .withColumn("Date", F.to_date(third_df["Date"], "dd-MM-yyyy")) \
                   .withColumn("Value", third_df["Value"].cast("float"))

# Show the transformed DataFrame
third_df_transformed.show()


+----------+---------+------------+--------------------+--------------------+---------------------+----------+-------+
| create_dt|metric_id|feature_name|             metrics|       display_names|operating_system_type|      Date|  Value|
+----------+---------+------------+--------------------+--------------------+---------------------+----------+-------+
|2025-02-06|        1|    BIL LING|DR_Visits to Bill...|Billing Section V...|            Apple iOS|2024-08-29|71742.0|
|2025-02-06|        2|    BIL LING|DR_Billing Make a...|       Make Payments|            Apple iOS|2024-08-29| 39.419|
|2025-02-06|        3|    BIL LING|DR_Client Errors ...|Client Errors on ...|            Apple iOS|2024-08-29|  0.004|
|2025-02-06|        4|    BIL LING|DR_Server Errors ...|Server Errors on ...|            Apple iOS|2024-08-29|  0.004|
|2025-02-06|        5|    BIL LING|   DR_Payment Submit|      Payment Submit|            Apple iOS|2024-08-29| 36.536|
|2025-02-06|        6|    BIL LING|         DR_P

In [18]:
third_df_transformed.printSchema()

root
 |-- create_dt: date (nullable = true)
 |-- metric_id: integer (nullable = true)
 |-- feature_name: string (nullable = true)
 |-- metrics: string (nullable = true)
 |-- display_names: string (nullable = true)
 |-- operating_system_type: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Value: float (nullable = true)


In [19]:
third_output_path = "s3://cci-dig-aicoe-data-sb/processed/Health_Score/healthscore_percent_both/"
third_df_transformed.coalesce(1).write.format("parquet").mode("append").save(third_output_path)

In [29]:
output_path = "s3://cci-dig-aicoe-data-sb/processed/Health_Score/healthscore_calculations/"
job_log_table_write_df = first_df_transformed.write.format("parquet").mode("append").save(output_path)

In [20]:
fourth_Table = "s3://cci-dig-aicoe-data-sb/processed/Health_Score/fourth_table/fourth_table.csv"

In [21]:
fourth_df = spark.read.csv(fourth_Table, header=True, inferSchema=True)
fourth_df.show(5)

+-------------------+---------+-------------+--------------------+-------------+---------------------+------------+------------+------+--------+-------+---------+-----------+------------+--------------------+---------------------+-----------------------+----------------+
|          create_dt|metric_id| feature_name|             metrics|display_names|operating_system_type|1st Quartile|3rd Quartile|   IQR|   Upper|  Lower|Yesterday|last_7_days|last_30_days|% Change Last 7 Days|% Change Last 30 Days|metric_nature_indicator|colour_indicator|
+-------------------+---------+-------------+--------------------+-------------+---------------------+------------+------------+------+--------+-------+---------+-----------+------------+--------------------+---------------------+-----------------------+----------------+
|2025-02-06 00:00:00|       22|   COX MOBILE|DR_Mobile Lock En...|         null|            Apple iOS|         0.0|       143.0| 143.0|  321.75|-178.75|     42.0|     50.714|      45.8

In [22]:
fourth_df.printSchema()

root
 |-- create_dt: timestamp (nullable = true)
 |-- metric_id: integer (nullable = true)
 |-- feature_name: string (nullable = true)
 |-- metrics: string (nullable = true)
 |-- display_names: string (nullable = true)
 |-- operating_system_type: string (nullable = true)
 |-- 1st Quartile: double (nullable = true)
 |-- 3rd Quartile: double (nullable = true)
 |-- IQR: double (nullable = true)
 |-- Upper: double (nullable = true)
 |-- Lower: double (nullable = true)
 |-- Yesterday: double (nullable = true)
 |-- last_7_days: double (nullable = true)
 |-- last_30_days: double (nullable = true)
 |-- % Change Last 7 Days: double (nullable = true)
 |-- % Change Last 30 Days: double (nullable = true)
 |-- metric_nature_indicator: string (nullable = true)
 |-- colour_indicator: string (nullable = true)


In [23]:
from pyspark.sql import functions as F

# Convert create_dt to DateType
fourth_df = fourth_df.withColumn("create_dt", F.to_date(fourth_df["create_dt"], "dd-MM-yyyy"))

# Convert last_7_days and last_30_days to DoubleType
fourth_df = fourth_df.withColumn("last_7_days", fourth_df["last_7_days"].cast("double"))
fourth_df = fourth_df.withColumn("last_30_days", fourth_df["last_30_days"].cast("double"))



# Show some of the data to verify
fourth_df.show()


+----------+---------+-------------+--------------------+--------------------+---------------------+------------+------------+------+--------+--------+---------+-----------+------------+--------------------+---------------------+-----------------------+----------------+
| create_dt|metric_id| feature_name|             metrics|       display_names|operating_system_type|1st Quartile|3rd Quartile|   IQR|   Upper|   Lower|Yesterday|last_7_days|last_30_days|% Change Last 7 Days|% Change Last 30 Days|metric_nature_indicator|colour_indicator|
+----------+---------+-------------+--------------------+--------------------+---------------------+------------+------------+------+--------+--------+---------+-----------+------------+--------------------+---------------------+-----------------------+----------------+
|2025-02-06|       22|   COX MOBILE|DR_Mobile Lock En...|                null|            Apple iOS|         0.0|       143.0| 143.0|  321.75| -178.75|     42.0|     50.714|      45.833| 

In [24]:
fourth_df.printSchema()

root
 |-- create_dt: date (nullable = true)
 |-- metric_id: integer (nullable = true)
 |-- feature_name: string (nullable = true)
 |-- metrics: string (nullable = true)
 |-- display_names: string (nullable = true)
 |-- operating_system_type: string (nullable = true)
 |-- 1st Quartile: double (nullable = true)
 |-- 3rd Quartile: double (nullable = true)
 |-- IQR: double (nullable = true)
 |-- Upper: double (nullable = true)
 |-- Lower: double (nullable = true)
 |-- Yesterday: double (nullable = true)
 |-- last_7_days: double (nullable = true)
 |-- last_30_days: double (nullable = true)
 |-- % Change Last 7 Days: double (nullable = true)
 |-- % Change Last 30 Days: double (nullable = true)
 |-- metric_nature_indicator: string (nullable = true)
 |-- colour_indicator: string (nullable = true)


In [25]:
fourth_output_path = "s3://cci-dig-aicoe-data-sb/processed/Health_Score/healthscore_calculations/"
fourth_df_table_write_df = fourth_df.write.format("parquet").mode("append").save(fourth_output_path)

In [1]:
#spark.sparkContext.getConf().getAll()
#for item in spark.sparkContext.getConf().getAll():
 #   print(item)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.7 
Trying to create a Glue session for the kernel.
Session Type: glueetl
Session ID: 30baa921-d157-40bb-acf8-429db30331ca
Applying the following default arguments:
--glue_kernel_version 1.0.7
--enable-glue-datacatalog true
Waiting for session 30baa921-d157-40bb-acf8-429db30331ca to get into ready status...
Session 30baa921-d157-40bb-acf8-429db30331ca has been created.



#### Example: Convert the DynamicFrame to a Spark DataFrame and display a sample of the data


In [6]:
day_aggregated_new = """
WITH filtered_auth AS (
    SELECT
        actor_alternateid AS user_id,
        CAST(event_date AS DATE) AS event_date,
        host,
        uuid,
        outcome_result
    FROM
        ciam.successful_authentications_okta
    WHERE
        CAST(event_date AS DATE) >= CURRENT_DATE - INTERVAL 1 YEAR  -- Fixed: Subtract 1 year from current date
),
filtered_signon AS (
    SELECT
        username AS user_id,
        application,
        CAST(event_date AS DATE) AS event_date
    FROM
        ciam.single_signon_okta
    WHERE
        CAST(event_date AS DATE) >= CURRENT_DATE - INTERVAL 1 YEAR  -- Fixed: Subtract 1 year from current date
),
aggregated_auth AS (
    SELECT
        user_id,
        event_date,
        host,
        COUNT(uuid) AS Authentication_Attempt,
        COUNT(DISTINCT CASE WHEN outcome_result = 'SUCCESS' THEN uuid END) AS authentication_success_result
    FROM
        filtered_auth
    GROUP BY
        user_id, event_date, host
),
aggregated_signon AS (
    SELECT
        user_id,
        event_date,
        application
    FROM
        filtered_signon
    GROUP BY
        user_id, event_date, application
)
SELECT
    a.event_date AS Event_Date,
    a.host AS Authentication_Host,
    o.application AS Authentication_Channel,
    a.Authentication_Attempt,
    a.authentication_success_result,
    cast(NULL as varchar(255)) AS Authentication_Error,  -- Fixed: Specify length for varchar
    'Login Credentials' AS Authentication_method,
    CASE
        WHEN m.eventtype = 'group.user_membership.add' THEN 'tsv enrolled'
        ELSE NULL
    END AS Activity_type
FROM
    aggregated_auth a LEFT JOIN edw.customer_guid_dtl_dim g ON a.user_id = g.user_id
LEFT JOIN
    aggregated_signon o ON g.user_id = o.user_id AND a.event_date = o.event_date
LEFT JOIN
    ciam.mfa_total_mfa_users m ON g.user_id = m.username
GROUP BY 
    a.event_date,
    a.host,
    o.application,
    a.Authentication_Attempt,
    a.authentication_success_result,
    CASE
        WHEN m.eventtype = 'group.user_membership.add' THEN 'tsv enrolled'
        ELSE NULL
    END
"""

In [7]:
day_aggregated_new_df = spark.sql(day_aggregated_new)
day_aggregated_new_df.show()

+----------+-------------------+----------------------+----------------------+-----------------------------+--------------------+---------------------+-------------+
|Event_Date|Authentication_Host|Authentication_Channel|Authentication_Attempt|authentication_success_result|Authentication_Error|Authentication_method|Activity_type|
+----------+-------------------+----------------------+----------------------+-----------------------------+--------------------+---------------------+-------------+
|2024-08-21|   cci-res.okta.com|               cox_app|                     3|                            3|                null|    Login Credentials|         null|
|2024-08-10|   cci-res.okta.com|                  null|                     1|                            1|                null|    Login Credentials| tsv enrolled|
|2024-11-14|   cci-res.okta.com|               cox_app|                     1|                            1|                null|    Login Credentials| tsv enrolled|
|202

In [8]:
day_aggregated_new_df.count()

205515


In [9]:
day_aggregated_new_df.persist(StorageLevel.MEMORY_AND_DISK)

DataFrame[Event_Date: date, Authentication_Host: string, Authentication_Channel: string, Authentication_Attempt: bigint, authentication_success_result: bigint, Authentication_Error: string, Authentication_method: string, Activity_type: string]


In [11]:
day_aggregated_new_df.printSchema()

root
 |-- Event_Date: date (nullable = true)
 |-- Authentication_Host: string (nullable = true)
 |-- Authentication_Channel: string (nullable = true)
 |-- Authentication_Attempt: long (nullable = false)
 |-- authentication_success_result: long (nullable = false)
 |-- Authentication_Error: string (nullable = true)
 |-- Authentication_method: string (nullable = false)
 |-- Activity_type: string (nullable = true)


In [43]:
day_aggregated_new_df.count()

207604


In [16]:
user_aggregated_new = """
WITH filtered_auth AS (
    SELECT
        actor_alternateid AS user_id,
        host,
        uuid,
        outcome_result,
        CAST(event_date AS DATE) AS event_date
    FROM
        ciam.successful_authentications_okta
    WHERE
        CAST(event_date AS DATE) >= DATE_ADD(CURRENT_DATE, -90)
),
filtered_signon AS (
    SELECT
        username AS user_id,
        application,
        CAST(event_date AS DATE) AS event_date
    FROM
        ciam.single_signon_okta
    WHERE
        CAST(event_date AS DATE) >= DATE_ADD(CURRENT_DATE, -90)
),
aggregated_auth AS (
    SELECT
        user_id,
        host,
        COUNT(uuid) AS Authentication_Attempt,
        COUNT(DISTINCT CASE WHEN outcome_result = 'SUCCESS' THEN uuid END) AS authentication_success_result
    FROM
        filtered_auth
    GROUP BY
        user_id, host
),
aggregated_signon AS (
    SELECT
        user_id,
        application
    FROM
        filtered_signon
    GROUP BY
        user_id, application
)
SELECT
    g.household_member_guid AS User_GUID,
    a.host AS Authentication_Host,
    o.application AS Authentication_Channel,
    a.Authentication_Attempt,
    a.authentication_success_result,
    cast(NULL as varchar(255)) AS Authentication_Error,
    'Login Credentials' AS authentication_method,
    CASE
        WHEN m.eventtype = 'group.user_membership.add' THEN 'tsv enrolled'
        ELSE NULL
    END AS activity_type
FROM
    aggregated_auth a
LEFT JOIN edw.customer_guid_dtl_dim g ON a.user_id = g.user_id
LEFT JOIN aggregated_signon o ON o.user_id = g.user_id
LEFT JOIN ciam.mfa_total_mfa_users m ON m.username = g.user_id
GROUP BY 
    g.household_member_guid,
    a.host,
    o.application,
    a.Authentication_Attempt,
    a.authentication_success_result,
    CASE
        WHEN m.eventtype = 'group.user_membership.add' THEN 'tsv enrolled'
        ELSE NULL
    END
"""

In [17]:
user_aggregated_new_df = spark.sql(user_aggregated_new)
user_aggregated_new_df.show()

+--------------------+-------------------+----------------------+----------------------+-----------------------------+--------------------+---------------------+-------------+
|           User_GUID|Authentication_Host|Authentication_Channel|Authentication_Attempt|authentication_success_result|Authentication_Error|authentication_method|activity_type|
+--------------------+-------------------+----------------------+----------------------+-----------------------------+--------------------+---------------------+-------------+
|d957a880-a424-11e...|   cci-res.okta.com|               cox_app|                     9|                            9|                null|    Login Credentials|         null|
|2a1e0a04-5267-6a8...|   cci-res.okta.com|             CoxDotCom|                    14|                           14|                null|    Login Credentials|         null|
|2a1e0a0e-99ab-129...|   cci-res.okta.com|        Panoramic Wifi|                     1|                            1|  

In [18]:
user_aggregated_new_df.count()

6257889


In [19]:
user_aggregated_new_df.persist(StorageLevel.MEMORY_AND_DISK)

DataFrame[User_GUID: string, Authentication_Host: string, Authentication_Channel: string, Authentication_Attempt: bigint, authentication_success_result: bigint, Authentication_Error: string, authentication_method: string, activity_type: string]


In [20]:
user_aggregated_new_df.printSchema()

root
 |-- User_GUID: string (nullable = true)
 |-- Authentication_Host: string (nullable = true)
 |-- Authentication_Channel: string (nullable = true)
 |-- Authentication_Attempt: long (nullable = false)
 |-- authentication_success_result: long (nullable = false)
 |-- Authentication_Error: string (nullable = true)
 |-- authentication_method: string (nullable = false)
 |-- activity_type: string (nullable = true)


In [7]:
user_aggregated_new_df.count()

6608324


In [10]:
adobe_fact_new = """
WITH web_contact_history AS (
    SELECT
        customer_key,
        evar61_coxcust_guid AS User_guid,
        evar75_marketing_cloud_id AS Adobe_ECID,
        visits AS Adobe_Visit_Id,
        date_time AS Activity_Date,
        pagename AS Activity_Page,
        prop10_custom_links as p10,
        prop15_previous_page as previous_page,
        mvvar3 AS Server_Error,
        evar46_page_name as activity_pagename,
        CAST(NULL AS varchar(255)) AS Client_Error,
        campaign AS Traffic_Source_Detail_sc_id
    FROM webanalytics.web_contact_history
    WHERE to_date(SUBSTR(CAST(dt AS STRING), 1, 10), 'yyyy-MM-dd') >= DATE_ADD(CURRENT_DATE, -30)
    AND (pagename LIKE 'cox:res:myprofile%' OR evar46_page_name LIKE 'cox:res:myprofile%')
),
app_contact_history AS (
    SELECT
        cast(customer_key as bigint) as customer_key,
        coxcust_guid_v61 AS User_guid,
        CAST(NULL AS varchar(255)) AS Adobe_ECID,
        visits AS Adobe_Visit_Id,
        date_time AS Activity_Date,
        pagename AS Activity_Page,
        custom_link_clicks_p10 as p10,
        previous_page_p15 as previous_page,
        server_form_error_p13 AS Server_Error,
        page_name_v46 as activity_pagename,
        CAST(NULL AS varchar(255)) AS Client_Error,
        post_evar40 AS Traffic_Source_Detail_sc_id
    FROM mobile_data_temp.app_contact_history
    WHERE to_date(SUBSTR(CAST(dt AS STRING), 1, 10), 'yyyy-MM-dd') >= DATE_ADD(CURRENT_DATE, -30)
    AND ((pagename LIKE 'coxapp:reg:%' OR pagename LIKE 'coxapp:myaccount%') OR (page_name_v46 LIKE 'coxapp:reg:%' OR page_name_v46 LIKE 'coxapp:myaccount%'))
),
union_cte AS (
    SELECT * FROM web_contact_history W
    UNION ALL 
    SELECT * FROM app_contact_history a
)
SELECT DISTINCT 
    u.customer_key,
    u.User_guid,
    u.Adobe_ECID,
    u.Adobe_Visit_Id,
    u.Activity_Date,
    a2.create_dt as Registration_Date,
    b.inception_dt as Inception_date,
    CASE 
        WHEN u.Activity_Page LIKE '%:communication%' THEN 'Manage Profile'
        WHEN u.Activity_Page LIKE '%:contacts%' THEN 'Manage Profile'
        WHEN u.Activity_Page LIKE '%:disability%' THEN 'Manage Profile'
        WHEN u.Activity_Page LIKE '%:confirmation%' THEN 'Email Verification'
        WHEN u.Activity_Page LIKE '%:forgot-password:%' THEN 'Forgot Password'
        WHEN u.Activity_Page LIKE '%:forgot-userid:%' THEN 'Forgot UserID'
        WHEN u.Activity_Page LIKE '%:forgotuserid:%' THEN 'Forgot UserID'
        WHEN u.Activity_Page LIKE '%:fuid:%' THEN 'Forgot UserID'
        WHEN u.Activity_Page LIKE '%:home%' THEN 'Manage Profile'
        WHEN u.Activity_Page LIKE '%:mailing-address%' THEN 'Manage Profile'
        WHEN u.Activity_Page LIKE '%:mailingaddress%' THEN 'Manage Profile'
        WHEN u.Activity_Page LIKE '%:manageusers%' THEN 'Manage Profile'
        WHEN u.Activity_Page LIKE '%:notifications%' THEN 'Manage Profile'
        WHEN u.Activity_Page LIKE '%:number-lock-protection%' THEN 'Number Lock Protection'
        WHEN u.Activity_Page LIKE '%:password%' THEN 'Manage Profile'
        WHEN u.Activity_Page LIKE '%:privacy%' THEN 'Manage Profile'
        WHEN u.Activity_Page LIKE '%:reg:%' THEN 'Registration'
        WHEN u.Activity_Page LIKE '%:security%' THEN 'Manage Profile'
        WHEN u.Activity_Page LIKE '%:updateprofile%' THEN 'Manage Profile'
        WHEN u.Activity_Page LIKE '%:syncupaccount%' THEN 'Sync Account'
        WHEN u.Activity_Page LIKE '%:tsv' THEN 'TSV Enrollment'
        WHEN u.Activity_Page LIKE '%:tsv:email:%' THEN 'TSV Verification'
        WHEN u.Activity_Page LIKE '%:tsv:call:%' THEN 'TSV Verification'
        WHEN u.Activity_Page LIKE '%:tsv:text:%' THEN 'TSV Verification'
        WHEN u.Activity_Page LIKE '%:tsv:reset:%' THEN 'TSV Reset'
        WHEN u.Activity_Page LIKE '%:verify-identity:%' THEN 'Verify Contact'
        ELSE 'Unknown'        
    END AS Activity_Name,
    CASE
        WHEN u.Activity_Page LIKE '%:email' then 'Attempt' 
        WHEN u.Activity_Page LIKE '%phone' then 'Attempt'  
        WHEN u.Activity_Page LIKE '%:phone:call' then 'Attempt' 
        WHEN u.Activity_Page LIKE '%:phone:text' then 'Attempt'
        WHEN u.Activity_Page LIKE '%lookup-account' then 'Attempt' 
        WHEN u.Activity_Page LIKE '%lookup-email' then 'Attempt' 
        WHEN u.Activity_Page LIKE '%lookup-phone' then 'Attempt' 
        WHEN u.Activity_Page LIKE '%lookup-address' then 'Attempt' 
        WHEN u.Activity_Page LIKE '%userid-exists' then 'Already Registered'
        WHEN u.Activity_Page LIKE '%multi-address%' then 'Multiple Address' 
        WHEN u.Activity_Page LIKE '%:notifications' then 'Prefrence Center'
        WHEN u.Activity_Page LIKE '%:communication' then 'Prefrence Center'
        WHEN u.Activity_Page LIKE '%:secret-question' then 'Secret Question'
        WHEN u.Activity_Page LIKE '%:sendcode' then 'Send Code'
        WHEN u.Activity_Page LIKE 'cox:res:myprofile:reg:userid-password' then 'Setup Credentials'
        WHEN u.Activity_Page LIKE '%:landing' then 'Start page' 
        WHEN u.Activity_Page LIKE '%:confirmation' then 'Success' 
        WHEN u.Activity_Page LIKE '%:lookup-account-success' then 'Success'
        WHEN u.Activity_Page LIKE '%:lookup-address-success' then 'Success'
        WHEN u.Activity_Page LIKE '%:success' then 'Success' 
        WHEN u.Activity_Page LIKE '%:verification%' then 'Verify Code' 
        WHEN u.Activity_Page LIKE '%:verifycode' then 'Verify Code' 
        ELSE 'Unknown'
    END AS Navigation_step,
    CASE 
        WHEN u.activity_pagename LIKE '%:communication%' THEN 'Manage Profile'
        WHEN u.activity_pagename LIKE '%:contacts%' THEN 'Manage Profile'
        WHEN u.activity_pagename LIKE '%:disability%' THEN 'Manage Profile'
        WHEN u.activity_pagename LIKE '%:confirmation%' THEN 'Email Verification'
        WHEN u.activity_pagename LIKE '%:forgot-password:%' THEN 'Forgot Password'
        WHEN u.activity_pagename LIKE '%:forgot-userid:%' THEN 'Forgot UserID'
        WHEN u.activity_pagename LIKE '%:forgotuserid:%' THEN 'Forgot UserID'
        WHEN u.activity_pagename LIKE '%:fuid:%' THEN 'Forgot UserID'
        WHEN u.activity_pagename LIKE '%:home%' THEN 'Manage Profile'
        WHEN u.activity_pagename LIKE '%:mailing-address%' THEN 'Manage Profile'
        WHEN u.activity_pagename LIKE '%:mailingaddress%' THEN 'Manage Profile'
        WHEN u.activity_pagename LIKE '%:manageusers%' THEN 'Manage Profile'
        WHEN u.activity_pagename LIKE '%:notifications%' THEN 'Manage Profile'
        WHEN u.activity_pagename LIKE '%:number-lock-protection%' THEN 'Number Lock Protection'
        WHEN u.activity_pagename LIKE '%:password%' THEN 'Manage Profile'
        WHEN u.activity_pagename LIKE '%:privacy%' THEN 'Manage Profile'
        WHEN u.activity_pagename LIKE '%:reg:%' THEN 'Registration'
        WHEN u.activity_pagename LIKE '%:security%' THEN 'Manage Profile'
        WHEN u.activity_pagename LIKE '%:updateprofile%' THEN 'Manage Profile'
        WHEN u.activity_pagename LIKE '%:syncupaccount%' THEN 'Sync Account'
        WHEN u.activity_pagename LIKE '%:tsv' THEN 'TSV Enrollment'
        WHEN u.activity_pagename LIKE '%:tsv:email:%' THEN 'TSV Verification'
        WHEN u.activity_pagename LIKE '%:tsv:call:%' THEN 'TSV Verification'
        WHEN u.activity_pagename LIKE '%:tsv:text:%' THEN 'TSV Verification'
        WHEN u.activity_pagename LIKE '%:tsv:reset:%' THEN 'TSV Reset'
        WHEN u.activity_pagename LIKE '%:verify-identity:%' THEN 'Verify Contact'
        ELSE 'Unknown'        
    END AS methods,
    CASE
        WHEN u.activity_pagename LIKE '%:email' then 'Attempt' 
        WHEN u.activity_pagename LIKE '%phone' then 'Attempt'  
        WHEN u.activity_pagename LIKE '%:phone:call' then 'Attempt' 
        WHEN u.activity_pagename LIKE '%:phone:text' then 'Attempt'
        WHEN u.activity_pagename LIKE '%lookup-account' then 'Attempt' 
        WHEN u.activity_pagename LIKE '%lookup-email' then 'Attempt' 
        WHEN u.activity_pagename LIKE '%lookup-phone' then 'Attempt' 
        WHEN u.activity_pagename LIKE '%lookup-address' then 'Attempt' 
        WHEN u.activity_pagename LIKE '%userid-exists' then 'Already Registered'
        WHEN u.activity_pagename LIKE '%multi-address%' then 'Multiple Address' 
        WHEN u.activity_pagename LIKE '%:notifications' then 'Prefrence Center'
        WHEN u.activity_pagename LIKE '%:communication' then 'Prefrence Center'
        WHEN u.activity_pagename LIKE '%:secret-question' then 'Secret Question'
        WHEN u.activity_pagename LIKE '%:sendcode' then 'Send Code'
        WHEN u.activity_pagename LIKE 'cox:res:myprofile:reg:userid-password' then 'Setup Credentials'
        WHEN u.activity_pagename LIKE '%:landing' then 'Start page' 
        WHEN u.activity_pagename LIKE '%:confirmation' then 'Success' 
        WHEN u.activity_pagename LIKE '%:lookup-account-success' then 'Success'
        WHEN u.activity_pagename LIKE '%:lookup-address-success' then 'Success'
        WHEN u.activity_pagename LIKE '%:success' then 'Success' 
        WHEN u.activity_pagename LIKE '%:verification%' then 'Verify Code' 
        WHEN u.activity_pagename LIKE '%:verifycode' then 'Verify Code' 
        ELSE 'Unknown'
    END AS Feature,
    u.Activity_Page,
    u.p10,
    u.previous_page,
    u.activity_pagename,
    u.Server_Error,
    u.Client_Error,
    u.Traffic_Source_Detail_sc_id
FROM union_cte u
LEFT JOIN edw.customer_guid_dtl_dim a2 ON u.User_guid = a2.household_member_guid
LEFT JOIN edw.customer_dim b ON a2.customer_key = b.customer_key
GROUP BY u.customer_key,
    u.User_guid,
    u.Adobe_ECID,
    u.Adobe_Visit_Id,
    u.Activity_Date,
    u.Activity_Page,
    a2.create_dt,
    b.inception_dt,
    u.p10,
    u.previous_page,
    u.activity_pagename,
    u.Server_Error,
    u.Client_Error,
    u.Traffic_Source_Detail_sc_id,
    CASE 
        WHEN u.Activity_Page LIKE '%:communication%' THEN 'Manage Profile'
        WHEN u.Activity_Page LIKE '%:contacts%' THEN 'Manage Profile'
        WHEN u.Activity_Page LIKE '%:disability%' THEN 'Manage Profile'
        WHEN u.Activity_Page LIKE '%:confirmation%' THEN 'Email Verification'
        WHEN u.Activity_Page LIKE '%:forgot-password:%' THEN 'Forgot Password'
        WHEN u.Activity_Page LIKE '%:forgot-userid:%' THEN 'Forgot UserID'
        WHEN u.Activity_Page LIKE '%:forgotuserid:%' THEN 'Forgot UserID'
        WHEN u.Activity_Page LIKE '%:fuid:%' THEN 'Forgot UserID'
        WHEN u.Activity_Page LIKE '%:home%' THEN 'Manage Profile'
        WHEN u.Activity_Page LIKE '%:mailing-address%' THEN 'Manage Profile'
        WHEN u.Activity_Page LIKE '%:mailingaddress%' THEN 'Manage Profile'
        WHEN u.Activity_Page LIKE '%:manageusers%' THEN 'Manage Profile'
        WHEN u.Activity_Page LIKE '%:notifications%' THEN 'Manage Profile'
        WHEN u.Activity_Page LIKE '%:number-lock-protection%' THEN 'Number Lock Protection'
        WHEN u.Activity_Page LIKE '%:password%' THEN 'Manage Profile'
        WHEN u.Activity_Page LIKE '%:privacy%' THEN 'Manage Profile'
        WHEN u.Activity_Page LIKE '%:reg:%' THEN 'Registration'
        WHEN u.Activity_Page LIKE '%:security%' THEN 'Manage Profile'
        WHEN u.Activity_Page LIKE '%:updateprofile%' THEN 'Manage Profile'
        WHEN u.Activity_Page LIKE '%:syncupaccount%' THEN 'Sync Account'
        WHEN u.Activity_Page LIKE '%:tsv' THEN 'TSV Enrollment'
        WHEN u.Activity_Page LIKE '%:tsv:email:%' THEN 'TSV Verification'
        WHEN u.Activity_Page LIKE '%:tsv:call:%' THEN 'TSV Verification'
        WHEN u.Activity_Page LIKE '%:tsv:text:%' THEN 'TSV Verification'
        WHEN u.Activity_Page LIKE '%:tsv:reset:%' THEN 'TSV Reset'
        WHEN u.Activity_Page LIKE '%:verify-identity:%' THEN 'Verify Contact'
        ELSE 'Unknown'        
    END,
    CASE
        WHEN u.Activity_Page LIKE '%:email' then 'Attempt' 
        WHEN u.Activity_Page LIKE '%phone' then 'Attempt'  
        WHEN u.Activity_Page LIKE '%:phone:call' then 'Attempt' 
        WHEN u.Activity_Page LIKE '%:phone:text' then 'Attempt'
        WHEN u.Activity_Page LIKE '%lookup-account' then 'Attempt' 
        WHEN u.Activity_Page LIKE '%lookup-email' then 'Attempt' 
        WHEN u.Activity_Page LIKE '%lookup-phone' then 'Attempt' 
        WHEN u.Activity_Page LIKE '%lookup-address' then 'Attempt' 
        WHEN u.Activity_Page LIKE '%userid-exists' then 'Already Registered'
        WHEN u.Activity_Page LIKE '%multi-address%' then 'Multiple Address' 
        WHEN u.Activity_Page LIKE '%:notifications' then 'Prefrence Center'
        WHEN u.Activity_Page LIKE '%:communication' then 'Prefrence Center'
        WHEN u.Activity_Page LIKE '%:secret-question' then 'Secret Question'
        WHEN u.Activity_Page LIKE '%:sendcode' then 'Send Code'
        WHEN u.Activity_Page LIKE 'cox:res:myprofile:reg:userid-password' then 'Setup Credentials'
        WHEN u.Activity_Page LIKE '%:landing' then 'Start page' 
        WHEN u.Activity_Page LIKE '%:confirmation' then 'Success' 
        WHEN u.Activity_Page LIKE '%:lookup-account-success' then 'Success'
        WHEN u.Activity_Page LIKE '%:lookup-address-success' then 'Success'
        WHEN u.Activity_Page LIKE '%:success' then 'Success' 
        WHEN u.Activity_Page LIKE '%:verification%' then 'Verify Code' 
        WHEN u.Activity_Page LIKE '%:verifycode' then 'Verify Code' 
        ELSE 'Unknown'
    END

"""

In [11]:
adobe_fact_new_df = spark.sql(adobe_fact_new)
adobe_fact_new_df.show()

+------------+--------------------+--------------------+--------------------+-------------------+-----------------+--------------+------------------+----------------+------------------+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+---------------------------+
|customer_key|           User_guid|          Adobe_ECID|      Adobe_Visit_Id|      Activity_Date|Registration_Date|Inception_date|     Activity_Name| Navigation_step|           methods|         Feature|       Activity_Page|                 p10|       previous_page|   activity_pagename|        Server_Error|Client_Error|Traffic_Source_Detail_sc_id|
+------------+--------------------+--------------------+--------------------+-------------------+-----------------+--------------+------------------+----------------+------------------+----------------+--------------------+--------------------+--------------------+--------------------+----------------

In [12]:
adobe_fact_new_df.count()

7275861


In [11]:
adobe_fact_new_df.persist(StorageLevel.MEMORY_AND_DISK)

DataFrame[User_guid: string, Adobe_ECID: string, Adobe_Visit_Id: string, Activity_Date: string, Registration_Date: timestamp, Inception_date: string, Activity_Name: string, Navigation_step: string, Activity_Page: string, Server_Error: string, Client_Error: string, Traffic_Source_Detail_sc_id: string]


In [12]:
adobe_fact_new_df.printSchema()

root
 |-- User_guid: string (nullable = true)
 |-- Adobe_ECID: string (nullable = true)
 |-- Adobe_Visit_Id: string (nullable = true)
 |-- Activity_Date: string (nullable = true)
 |-- Registration_Date: timestamp (nullable = true)
 |-- Inception_date: string (nullable = true)
 |-- Activity_Name: string (nullable = false)
 |-- Navigation_step: string (nullable = false)
 |-- Activity_Page: string (nullable = true)
 |-- Server_Error: string (nullable = true)
 |-- Client_Error: string (nullable = true)
 |-- Traffic_Source_Detail_sc_id: string (nullable = true)


In [13]:
adobe_fact_new_df.count()

128572780


In [2]:
account_dim_sum_new_1 = """
WITH customer_data AS (
    SELECT DISTINCT c.customer_key,
           CAST(c.account_nbr AS varchar(255)) AS account_nbr,
           c.site_id,
           c.res_comm_ind,
           c.customer_status_cd,
           CAST(c.account_guid AS varchar(255)) AS account_guid,
           CAST(c.prim_account_holder_guid AS varchar(255)) AS prim_account_holder_guid,
           c.test_account_key AS test_account_key,
           c.inception_dt
    FROM edw.customer_dim c
    WHERE test_account_key = 2
),
revenue_data AS (
    SELECT r.customer_key,
           r.site_id,
           r.dwelling_type_key,
           r.easy_pay_flag,
           CASE WHEN r.mobile_gross_mrc > 0 THEN '1' ELSE '0' END AS Mobile_Flag,
           r.customer_substatus_key,
           r.time_key
    FROM edw.customer_revenue_fact r
    WHERE r.bill_type_key != 2 
          AND to_date(r.time_key, 'yyyy-MM-dd') >= add_months(CURRENT_DATE, -1)
),
customer_status AS (
    SELECT customer_substatus_key, sub_status_desc 
    FROM edw.customer_substatus_dim
),
dwelling_data AS (
    SELECT DISTINCT d.dwelling_type_key,
           d.dwelling_type_desc
    FROM edw.dwelling_type_dim d
),
account_summary AS (
    SELECT s.customer_key,
           s.employee_flag,
           s.data_flag,
           s.site_id,
           s.cable_flag,
           s.wireless_flag,
           s.do_not_call_flag,
           s.do_not_email_flag,
           s.do_not_mail_flag,
           s.telephony_flag,
           s.do_not_market_flag,
           s.time_key,
           ROW_NUMBER() OVER (PARTITION BY s.customer_key ORDER BY s.time_key DESC) AS rn
    FROM edw.cust_acct_sum s
),
guid_data AS (
    SELECT g.customer_key,
           g.create_dt,
           g.household_member_guid,
           ROW_NUMBER() OVER (PARTITION BY g.customer_key ORDER BY g.create_dt ASC) AS rn
    FROM edw.customer_guid_dtl_dim g
),
web_contact AS (
    SELECT campaign,
           evar61_coxcust_guid,
           ROW_NUMBER() OVER (PARTITION BY evar61_coxcust_guid ORDER BY MIN(dt)) AS rn
    FROM webanalytics.web_contact_history
    WHERE visits IN (
        SELECT DISTINCT visits
        FROM webanalytics.web_contact_history
        WHERE pagename = 'cox:res:myprofile:reg:confirmation'
    )
    GROUP BY campaign, evar61_coxcust_guid
),
app_contact AS (
    SELECT coxcust_guid_v61,
           post_evar40,
           ROW_NUMBER() OVER (PARTITION BY coxcust_guid_v61 ORDER BY MIN(dt)) AS rn
    FROM mobile_data_temp.app_contact_history
    WHERE visits IN (
        SELECT DISTINCT visits
        FROM mobile_data_temp.app_contact_history
        WHERE pagename = 'coxapp:reg:confirmation'
    )
    GROUP BY coxcust_guid_v61, post_evar40
),
ivr_contact AS (
    SELECT DISTINCT i.customer_key,
           DATE_FORMAT(CAST(i.time_key AS DATE), 'yyyy-MM') AS Contact_Month,
           MAX(CAST(i.time_key AS DATE)) AS Last_Contacted_Date_IVR_Call
    FROM call.call_ivr_fact i
    GROUP BY i.customer_key, DATE_FORMAT(CAST(i.time_key AS DATE), 'yyyy-MM')
),
web_data AS (
    SELECT DISTINCT d.customer_key,
           DATE_FORMAT(CAST(d.dt AS DATE), 'yyyy-MM') AS Contact_Month,
           MAX(CAST(d.dt AS DATE)) AS Last_Contacted_Date_Cox_com
    FROM webanalytics.web_contact_history d
    GROUP BY d.customer_key, DATE_FORMAT(CAST(d.dt AS DATE), 'yyyy-MM')
),
mob_data AS (
    SELECT DISTINCT mob.customer_key,
           DATE_FORMAT(CAST(mob.dt AS DATE), 'yyyy-MM') AS Contact_Month,
           MAX(CAST(mob.dt AS DATE)) AS Last_Contacted_Date_Cox_App
    FROM mobile_data_temp.app_contact_history mob
    GROUP BY mob.customer_key, DATE_FORMAT(CAST(mob.dt AS DATE), 'yyyy-MM')
)
SELECT DISTINCT
    r.customer_key AS Customer_Key,
    c.account_nbr AS Account_Nbr,
    r.site_id AS Site_Id,
    c.res_comm_ind AS Res_Com_Ind,
    c.customer_status_cd AS Customer_Status,
    d.dwelling_type_desc AS House_Type,
    c.account_guid AS Account_GUID,
    c.prim_account_holder_guid AS User_GUID_Primary,
    s.employee_flag AS Employee_Flag,
    c.test_account_key AS Test_Account_Flag,
    c.inception_dt AS Inception_Date,
    CAST(NULL AS varchar(255)) AS Sale_Acquisition_Channel,
    g.create_dt AS Registration_Date,
    COALESCE(web.campaign, app.post_evar40) AS Registration_Traffic_Source_Detail,
    CASE
        WHEN LOWER(COALESCE(web.campaign, app.post_evar40)) LIKE 'cr_em_cns_ocall_event255' THEN 'Email-Order'
        WHEN LOWER(COALESCE(web.campaign, app.post_evar40)) LIKE 'cr_em_z_acct_onb%' THEN 'Email-Onboarding'
        WHEN LOWER(COALESCE(web.campaign, app.post_evar40)) LIKE 'cr_em%' THEN 'Email-Others'
        WHEN LOWER(COALESCE(web.campaign, app.post_evar40)) LIKE 'cr_sms%' THEN 'sms'
        WHEN LOWER(COALESCE(web.campaign, app.post_evar40)) LIKE 'cr_dm%' THEN 'direct mail'
        WHEN SUBSTRING(LOWER(COALESCE(web.campaign, app.post_evar40)), LENGTH(COALESCE(web.campaign, app.post_evar40)) - 5, 6) = 'vanity' THEN 'Vanity URL'
        WHEN LOWER(COALESCE(web.campaign, app.post_evar40)) LIKE '%panoapp%' THEN 'panoapp'
        WHEN COALESCE(web.campaign, app.post_evar40) IS NOT NULL AND LENGTH(COALESCE(web.campaign, app.post_evar40)) > 0 THEN 'organic'
        ELSE 'null'  
    END AS Registration_Traffic_Source,
    s.data_flag AS Data_Flag,
    s.cable_flag AS TV_Flag,
    s.telephony_flag AS Phone_Flag,
    CAST(NULL AS varchar(255)) AS Homelife_Flag,
    r.Mobile_Flag,
    cd.sub_status_desc,
    CAST(NULL AS varchar(255)) AS Pano_Flag,
    CAST(NULL AS varchar(255)) AS Pano_Device,
    r.easy_pay_flag AS Easy_Pay_Flag,
    s.do_not_call_flag AS Do_Not_Call_Flag,
    s.do_not_email_flag AS Do_Not_Email_Flag,
    s.do_not_mail_flag AS Do_Not_Mail_Flag,
    s.do_not_market_flag AS Do_Not_Market_Flag,
    MAX(ivr.Last_Contacted_Date_IVR_Call) AS Last_Contacted_Date_IVR_Call,
    MAX(w.Last_Contacted_Date_Cox_com) AS Last_Contacted_Date_Cox_com,
    MAX(mob.Last_Contacted_Date_Cox_App) AS Last_Contacted_Date_Cox_App, 
    CAST(NULL AS varchar(255)) AS Cox_Segment,
    CAST(NULL AS varchar(255)) AS Demographic_Info1,
    CAST(NULL AS varchar(255)) AS Demographic_Info2,
    r.time_key
FROM revenue_data r
LEFT JOIN dwelling_data d ON r.dwelling_type_key = d.dwelling_type_key
LEFT JOIN (SELECT * FROM account_summary WHERE rn = 1) s ON r.customer_key = CAST(s.customer_key AS double)
LEFT JOIN customer_status cd ON r.customer_substatus_key = cd.customer_substatus_key 
LEFT JOIN customer_data c ON r.customer_key = CAST(c.customer_key AS double)
LEFT JOIN (SELECT * FROM guid_data WHERE rn = 1) g ON r.customer_key = CAST(g.customer_key AS double)
LEFT JOIN (SELECT * FROM web_contact WHERE rn = 1) web ON g.household_member_guid = web.evar61_coxcust_guid
LEFT JOIN (SELECT * FROM app_contact WHERE rn = 1) app ON g.household_member_guid = app.coxcust_guid_v61
LEFT JOIN ivr_contact ivr ON r.customer_key = CAST(ivr.customer_key AS double) AND ivr.Contact_Month <= r.time_key
LEFT JOIN web_data w ON r.customer_key = CAST(w.customer_key AS double) AND w.Contact_Month <= r.time_key
LEFT JOIN mob_data mob ON r.customer_key = CAST(mob.customer_key AS double) AND mob.Contact_Month <= r.time_key
GROUP BY
    r.customer_key,
    c.account_nbr,
    r.site_id,
    c.res_comm_ind,
    c.customer_status_cd,
    d.dwelling_type_desc,
    c.account_guid,
    c.prim_account_holder_guid,
    s.employee_flag,
    c.test_account_key,
    c.inception_dt,
    g.create_dt,
    COALESCE(web.campaign, app.post_evar40),
    s.data_flag,
    s.cable_flag,
    s.telephony_flag,
    r.Mobile_Flag,
    cd.sub_status_desc,
    r.easy_pay_flag,
    s.do_not_call_flag,
    s.do_not_email_flag,
    s.do_not_mail_flag,
    s.do_not_market_flag,
    r.time_key
"""

In [3]:
account_dim_sum_new_1_df = spark.sql(account_dim_sum_new_1)
account_dim_sum_new_1_df.show()

+------------+------------+-------+-----------+---------------+-----------+--------------------+--------------------+-------------+-----------------+--------------------+------------------------+-------------------+----------------------------------+---------------------------+---------+-------+----------+-------------+-----------+---------------+---------+-----------+-------------+----------------+-----------------+----------------+------------------+----------------------------+---------------------------+---------------------------+-----------+-----------------+-----------------+----------+
|Customer_Key| Account_Nbr|Site_Id|Res_Com_Ind|Customer_Status| House_Type|        Account_GUID|   User_GUID_Primary|Employee_Flag|Test_Account_Flag|      Inception_Date|Sale_Acquisition_Channel|  Registration_Date|Registration_Traffic_Source_Detail|Registration_Traffic_Source|Data_Flag|TV_Flag|Phone_Flag|Homelife_Flag|Mobile_Flag|sub_status_desc|Pano_Flag|Pano_Device|Easy_Pay_Flag|Do_Not_Call_Fl

In [4]:
account_dim_sum_new_1_df.count()

12262851


In [5]:
account_dim_sum_new_1_df.persist(StorageLevel.MEMORY_AND_DISK)

DataFrame[Customer_Key: double, Account_Nbr: string, Site_Id: smallint, Res_Com_Ind: string, Customer_Status: string, House_Type: string, Account_GUID: string, User_GUID_Primary: string, Employee_Flag: tinyint, Test_Account_Flag: double, Inception_Date: string, Sale_Acquisition_Channel: string, Registration_Date: timestamp, Registration_Traffic_Source_Detail: string, Registration_Traffic_Source: string, Data_Flag: tinyint, TV_Flag: tinyint, Phone_Flag: tinyint, Homelife_Flag: string, Mobile_Flag: string, sub_status_desc: string, Pano_Flag: string, Pano_Device: string, Easy_Pay_Flag: string, Do_Not_Call_Flag: tinyint, Do_Not_Email_Flag: tinyint, Do_Not_Mail_Flag: tinyint, Do_Not_Market_Flag: tinyint, Last_Contacted_Date_IVR_Call: date, Last_Contacted_Date_Cox_com: date, Last_Contacted_Date_Cox_App: date, Cox_Segment: string, Demographic_Info1: string, Demographic_Info2: string, time_key: string]


In [6]:
account_dim_sum_new_1_df.printSchema()

root
 |-- Customer_Key: double (nullable = true)
 |-- Account_Nbr: string (nullable = true)
 |-- Site_Id: short (nullable = true)
 |-- Res_Com_Ind: string (nullable = true)
 |-- Customer_Status: string (nullable = true)
 |-- House_Type: string (nullable = true)
 |-- Account_GUID: string (nullable = true)
 |-- User_GUID_Primary: string (nullable = true)
 |-- Employee_Flag: byte (nullable = true)
 |-- Test_Account_Flag: double (nullable = true)
 |-- Inception_Date: string (nullable = true)
 |-- Sale_Acquisition_Channel: string (nullable = true)
 |-- Registration_Date: timestamp (nullable = true)
 |-- Registration_Traffic_Source_Detail: string (nullable = true)
 |-- Registration_Traffic_Source: string (nullable = false)
 |-- Data_Flag: byte (nullable = true)
 |-- TV_Flag: byte (nullable = true)
 |-- Phone_Flag: byte (nullable = true)
 |-- Homelife_Flag: string (nullable = true)
 |-- Mobile_Flag: string (nullable = false)
 |-- sub_status_desc: string (nullable = true)
 |-- Pano_Flag: strin

In [7]:
account_dim_sum_new_1_df.count()

12262851


In [16]:
account_dim_sum_new = """
WITH customer_data AS (
    SELECT DISTINCT c.customer_key,
           CAST(c.account_nbr AS varchar(255)) AS account_nbr,
           c.site_id,
           c.res_comm_ind,
           c.customer_status_cd,
           CAST(c.account_guid AS varchar(255)) AS account_guid,
           CAST(c.prim_account_holder_guid AS varchar(255)) AS prim_account_holder_guid,
           c.test_account_key AS test_account_key,
           c.inception_dt
    FROM edw.customer_dim c
    WHERE test_account_key = 2
),
revenue_data AS (
    SELECT r.customer_key,
           r.site_id,
           r.dwelling_type_key,
           r.easy_pay_flag,
           CASE WHEN r.mobile_gross_mrc > 0 THEN '1' ELSE '0' END AS Mobile_Flag,
           r.customer_substatus_key,
           r.time_key
    FROM edw.customer_revenue_fact r
    WHERE r.bill_type_key != 2 
          AND to_date(r.time_key, 'yyyy-MM-dd') >= add_months(CURRENT_DATE, -13)
),
customer_status AS (
    SELECT customer_substatus_key, sub_status_desc 
    FROM edw.customer_substatus_dim
),
dwelling_data AS (
    SELECT DISTINCT d.dwelling_type_key,
           d.dwelling_type_desc
    FROM edw.dwelling_type_dim d
),
account_summary AS (
    SELECT s.customer_key,
           s.employee_flag,
           s.data_flag,
           s.site_id,
           s.cable_flag,
           s.wireless_flag,
           s.do_not_call_flag,
           s.do_not_email_flag,
           s.do_not_mail_flag,
           s.telephony_flag,
           s.do_not_market_flag,
           s.time_key,
           ROW_NUMBER() OVER (PARTITION BY s.customer_key ORDER BY s.time_key DESC) AS rn
    FROM edw.cust_acct_sum s
),
guid_data AS (
    SELECT g.customer_key,
           g.create_dt,
           g.household_member_guid,
           ROW_NUMBER() OVER (PARTITION BY g.customer_key ORDER BY g.create_dt ASC) AS rn
    FROM edw.customer_guid_dtl_dim g
),
web_contact AS (
    SELECT campaign,
           evar61_coxcust_guid,
           ROW_NUMBER() OVER (PARTITION BY evar61_coxcust_guid ORDER BY MIN(dt)) AS rn
    FROM webanalytics.web_contact_history
    WHERE visits IN (
        SELECT DISTINCT visits
        FROM webanalytics.web_contact_history
        WHERE pagename = 'cox:res:myprofile:reg:confirmation'
    )
    GROUP BY campaign, evar61_coxcust_guid
),
app_contact AS (
    SELECT coxcust_guid_v61,
           post_evar40,
           ROW_NUMBER() OVER (PARTITION BY coxcust_guid_v61 ORDER BY MIN(dt)) AS rn
    FROM mobile_data_temp.app_contact_history
    WHERE visits IN (
        SELECT DISTINCT visits
        FROM mobile_data_temp.app_contact_history
        WHERE pagename = 'coxapp:reg:confirmation'
    )
    GROUP BY coxcust_guid_v61, post_evar40
),
ivr_contact AS (
    SELECT DISTINCT i.customer_key,
           DATE_FORMAT(CAST(i.time_key AS DATE), 'yyyy-MM') AS Contact_Month,
           MAX(CAST(i.time_key AS DATE)) AS Last_Contacted_Date_IVR_Call
    FROM call.call_ivr_fact i
    GROUP BY i.customer_key, DATE_FORMAT(CAST(i.time_key AS DATE), 'yyyy-MM')
),
web_data AS (
    SELECT DISTINCT d.customer_key,
           DATE_FORMAT(CAST(d.dt AS DATE), 'yyyy-MM') AS Contact_Month,
           MAX(CAST(d.dt AS DATE)) AS Last_Contacted_Date_Cox_com
    FROM webanalytics.web_contact_history d
    GROUP BY d.customer_key, DATE_FORMAT(CAST(d.dt AS DATE), 'yyyy-MM')
),
mob_data AS (
    SELECT DISTINCT mob.customer_key,
           DATE_FORMAT(CAST(mob.dt AS DATE), 'yyyy-MM') AS Contact_Month,
           MAX(CAST(mob.dt AS DATE)) AS Last_Contacted_Date_Cox_App
    FROM mobile_data_temp.app_contact_history mob
    GROUP BY mob.customer_key, DATE_FORMAT(CAST(mob.dt AS DATE), 'yyyy-MM')
)
SELECT DISTINCT
    r.customer_key AS Customer_Key,
    c.account_nbr AS Account_Nbr,
    r.site_id AS Site_Id,
    c.res_comm_ind AS Res_Com_Ind,
    c.customer_status_cd AS Customer_Status,
    d.dwelling_type_desc AS House_Type,
    c.account_guid AS Account_GUID,
    c.prim_account_holder_guid AS User_GUID_Primary,
    s.employee_flag AS Employee_Flag,
    c.test_account_key AS Test_Account_Flag,
    c.inception_dt AS Inception_Date,
    CAST(NULL AS varchar(255)) AS Sale_Acquisition_Channel,
    g.create_dt AS Registration_Date,
    COALESCE(web.campaign, app.post_evar40) AS Registration_Traffic_Source_Detail,
    CASE
        WHEN LOWER(COALESCE(web.campaign, app.post_evar40)) LIKE 'cr_em_cns_ocall_event255' THEN 'Email-Order'
        WHEN LOWER(COALESCE(web.campaign, app.post_evar40)) LIKE 'cr_em_z_acct_onb%' THEN 'Email-Onboarding'
        WHEN LOWER(COALESCE(web.campaign, app.post_evar40)) LIKE 'cr_em%' THEN 'Email-Others'
        WHEN LOWER(COALESCE(web.campaign, app.post_evar40)) LIKE 'cr_sms%' THEN 'sms'
        WHEN LOWER(COALESCE(web.campaign, app.post_evar40)) LIKE 'cr_dm%' THEN 'direct mail'
        WHEN SUBSTRING(LOWER(COALESCE(web.campaign, app.post_evar40)), LENGTH(COALESCE(web.campaign, app.post_evar40)) - 5, 6) = 'vanity' THEN 'Vanity URL'
        WHEN LOWER(COALESCE(web.campaign, app.post_evar40)) LIKE '%panoapp%' THEN 'panoapp'
        WHEN COALESCE(web.campaign, app.post_evar40) IS NOT NULL AND LENGTH(COALESCE(web.campaign, app.post_evar40)) > 0 THEN 'organic'
        ELSE 'null'  
    END AS Registration_Traffic_Source,
    s.data_flag AS Data_Flag,
    s.cable_flag AS TV_Flag,
    s.telephony_flag AS Phone_Flag,
    CAST(NULL AS varchar(255)) AS Homelife_Flag,
    r.Mobile_Flag,
    cd.sub_status_desc,
    CAST(NULL AS varchar(255)) AS Pano_Flag,
    CAST(NULL AS varchar(255)) AS Pano_Device,
    r.easy_pay_flag AS Easy_Pay_Flag,
    s.do_not_call_flag AS Do_Not_Call_Flag,
    s.do_not_email_flag AS Do_Not_Email_Flag,
    s.do_not_mail_flag AS Do_Not_Mail_Flag,
    s.do_not_market_flag AS Do_Not_Market_Flag,
    MAX(ivr.Last_Contacted_Date_IVR_Call) AS Last_Contacted_Date_IVR_Call,
    MAX(w.Last_Contacted_Date_Cox_com) AS Last_Contacted_Date_Cox_com,
    MAX(mob.Last_Contacted_Date_Cox_App) AS Last_Contacted_Date_Cox_App, 
    CAST(NULL AS varchar(255)) AS Cox_Segment,
    CAST(NULL AS varchar(255)) AS Demographic_Info1,
    CAST(NULL AS varchar(255)) AS Demographic_Info2,
    r.time_key
FROM revenue_data r
LEFT JOIN dwelling_data d ON r.dwelling_type_key = d.dwelling_type_key
LEFT JOIN (SELECT * FROM account_summary WHERE rn = 1) s ON r.customer_key = CAST(s.customer_key AS double)
LEFT JOIN customer_status cd ON r.customer_substatus_key = cd.customer_substatus_key 
LEFT JOIN customer_data c ON r.customer_key = CAST(c.customer_key AS double)
LEFT JOIN (SELECT * FROM guid_data WHERE rn = 1) g ON r.customer_key = CAST(g.customer_key AS double)
LEFT JOIN (SELECT * FROM web_contact WHERE rn = 1) web ON g.household_member_guid = web.evar61_coxcust_guid
LEFT JOIN (SELECT * FROM app_contact WHERE rn = 1) app ON g.household_member_guid = app.coxcust_guid_v61
LEFT JOIN ivr_contact ivr ON r.customer_key = CAST(ivr.customer_key AS double) AND ivr.Contact_Month <= r.time_key
LEFT JOIN web_data w ON r.customer_key = CAST(w.customer_key AS double) AND w.Contact_Month <= r.time_key
LEFT JOIN mob_data mob ON r.customer_key = CAST(mob.customer_key AS double) AND mob.Contact_Month <= r.time_key
GROUP BY
    r.customer_key,
    c.account_nbr,
    r.site_id,
    c.res_comm_ind,
    c.customer_status_cd,
    d.dwelling_type_desc,
    c.account_guid,
    c.prim_account_holder_guid,
    s.employee_flag,
    c.test_account_key,
    c.inception_dt,
    g.create_dt,
    COALESCE(web.campaign, app.post_evar40),
    s.data_flag,
    s.cable_flag,
    s.telephony_flag,
    r.Mobile_Flag,
    cd.sub_status_desc,
    r.easy_pay_flag,
    s.do_not_call_flag,
    s.do_not_email_flag,
    s.do_not_mail_flag,
    s.do_not_market_flag,
    r.time_key
"""

In [17]:
account_dim_sum_new_df = spark.sql(account_dim_sum_new)
account_dim_sum_new_df.show()

+------------+------------+-------+-----------+---------------+----------+--------------------+--------------------+-------------+-----------------+--------------------+------------------------+-------------------+----------------------------------+---------------------------+---------+-------+----------+-------------+-----------+---------------+---------+-----------+-------------+----------------+-----------------+----------------+------------------+----------------------------+---------------------------+---------------------------+-----------+-----------------+-----------------+----------+
|Customer_Key| Account_Nbr|Site_Id|Res_Com_Ind|Customer_Status|House_Type|        Account_GUID|   User_GUID_Primary|Employee_Flag|Test_Account_Flag|      Inception_Date|Sale_Acquisition_Channel|  Registration_Date|Registration_Traffic_Source_Detail|Registration_Traffic_Source|Data_Flag|TV_Flag|Phone_Flag|Homelife_Flag|Mobile_Flag|sub_status_desc|Pano_Flag|Pano_Device|Easy_Pay_Flag|Do_Not_Call_Flag

In [18]:
account_dim_sum_new_df.count()

87876227


In [19]:
account_dim_sum_new_df.persist(StorageLevel.MEMORY_AND_DISK)

DataFrame[Customer_Key: double, Account_Nbr: string, Site_Id: smallint, Res_Com_Ind: string, Customer_Status: string, House_Type: string, Account_GUID: string, User_GUID_Primary: string, Employee_Flag: tinyint, Test_Account_Flag: double, Inception_Date: string, Sale_Acquisition_Channel: string, Registration_Date: timestamp, Registration_Traffic_Source_Detail: string, Registration_Traffic_Source: string, Data_Flag: tinyint, TV_Flag: tinyint, Phone_Flag: tinyint, Homelife_Flag: string, Mobile_Flag: string, sub_status_desc: string, Pano_Flag: string, Pano_Device: string, Easy_Pay_Flag: string, Do_Not_Call_Flag: tinyint, Do_Not_Email_Flag: tinyint, Do_Not_Mail_Flag: tinyint, Do_Not_Market_Flag: tinyint, Last_Contacted_Date_IVR_Call: date, Last_Contacted_Date_Cox_com: date, Last_Contacted_Date_Cox_App: date, Cox_Segment: string, Demographic_Info1: string, Demographic_Info2: string, time_key: string]


In [20]:
account_dim_sum_new_df.printSchema()

root
 |-- Customer_Key: double (nullable = true)
 |-- Account_Nbr: string (nullable = true)
 |-- Site_Id: short (nullable = true)
 |-- Res_Com_Ind: string (nullable = true)
 |-- Customer_Status: string (nullable = true)
 |-- House_Type: string (nullable = true)
 |-- Account_GUID: string (nullable = true)
 |-- User_GUID_Primary: string (nullable = true)
 |-- Employee_Flag: byte (nullable = true)
 |-- Test_Account_Flag: double (nullable = true)
 |-- Inception_Date: string (nullable = true)
 |-- Sale_Acquisition_Channel: string (nullable = true)
 |-- Registration_Date: timestamp (nullable = true)
 |-- Registration_Traffic_Source_Detail: string (nullable = true)
 |-- Registration_Traffic_Source: string (nullable = false)
 |-- Data_Flag: byte (nullable = true)
 |-- TV_Flag: byte (nullable = true)
 |-- Phone_Flag: byte (nullable = true)
 |-- Homelife_Flag: string (nullable = true)
 |-- Mobile_Flag: string (nullable = false)
 |-- sub_status_desc: string (nullable = true)
 |-- Pano_Flag: strin

In [21]:
account_dim_sum_new_df.count()

87876227


In [2]:
profile_sum_dim_new_1 = """
WITH customer_dim AS (
    SELECT 
        b.customer_key AS Customer_Key,
        b.account_nbr AS Account_Nbr,
        b.site_id AS Site_Id,
        b.customer_status_cd AS Customer_Status,
        b.inception_dt AS Inception_Date,
        b.res_comm_ind AS Res_Com_Ind,
        b.test_account_key 
    FROM
        edw.customer_dim b
        WHERE test_account_key=2
),
revenue_data AS (
    SELECT r.customer_key,
           r.site_id,
           MAX(last_logged_in_date_okta) AS last_logged_in_date_okta,
           r.time_key
    FROM edw.customer_revenue_fact r 
    LEFT JOIN (
        SELECT DISTINCT
            a.Customer_Key,
            a.household_member_guid AS User_GUID,
            DATE_FORMAT(CAST(o.event_date AS DATE), 'yyyy-MM') AS Contact_Month,
            MAX(o.event_date) AS last_logged_in_date_okta
         FROM
            edw.customer_guid_dtl_dim a 
            LEFT JOIN ciam.successful_authentications_okta o 
            ON a.user_id = o.actor_alternateid
            GROUP BY
            a.Customer_Key, a.household_member_guid, DATE_FORMAT(CAST(o.event_date AS DATE), 'yyyy-MM')
    ) o 
    ON r.customer_key = o.customer_key 
    AND o.Contact_Month <= r.time_key
    WHERE r.bill_type_key != 2 
    AND to_date(r.time_key, 'yyyy-MM-dd') >= add_months(CURRENT_DATE, -1)
    GROUP BY r.customer_key,
             r.site_id,
             r.time_key
),
guid_data AS (
    SELECT 
        a.customer_key,
        a.create_dt AS Registration_date,
        a.user_id,
        a.prim_customer_flag AS Primary_Flag,
        a.household_member_guid AS User_GUID
    FROM
        edw.customer_guid_dtl_dim a
),
tsv_data AS (
    SELECT 
        a.customer_key,
        a.household_member_guid AS User_GUID,
        CASE WHEN (t.`household member guid`) IS NOT NULL THEN 'group.user_membership.add'
             ELSE NULL
        END AS TSV_Enrolled_Status
    FROM
        edw.customer_guid_dtl_dim a 
    INNER JOIN ciam_datamodel.tsv_guid_data t 
        ON cast(a.household_member_guid AS varchar(255)) = cast(t.`household member guid` AS varchar(255))
),
mfa_data AS (
    SELECT DISTINCT
        d.username,
        MAX(CASE
            WHEN d.outcome_reason LIKE '%EMAIL%'
                 AND d.event_date = sub.latest_event_date THEN 1
            ELSE 0
        END) AS TSV_EMAIL_Flag,
        MAX(CASE
            WHEN d.outcome_reason LIKE '%CALL%'
                 AND d.event_date = sub.latest_event_date THEN 1
            ELSE 0
        END) AS TSV_CALL_Flag,
        MAX(CASE
            WHEN d.outcome_reason LIKE '%SMS%'
                 AND d.event_date = sub.latest_event_date THEN 1
            ELSE 0
        END) AS TSV_SMS_Flag
    FROM
        ciam.mfa_factors_enrolled d
    LEFT JOIN (
        SELECT
            username AS user_id,
            outcome_reason,
            MAX(event_date) AS latest_event_date
        FROM
            ciam.mfa_factors_enrolled
        WHERE
            outcome_result = 'SUCCESS'
        GROUP BY
            username, outcome_reason
    ) sub
    ON d.username = sub.user_id AND d.outcome_reason = sub.outcome_reason
    WHERE
        d.outcome_result = 'SUCCESS'
    GROUP BY d.username
),
mob_data AS (
    SELECT
        mob.customer_key,
        mob.post_mobileappid,
        CASE
            WHEN mob.post_mobileappid LIKE 'CoxAccount%' THEN 'iOS'
            WHEN mob.post_mobileappid LIKE 'Cox %' THEN 'Android'
            ELSE 'Null'
        END AS Last_Logged_In_OS_Cox_App
    FROM
        mobile_data_temp.app_contact_history mob
)
SELECT DISTINCT a.User_GUID,
    b.Res_Com_Ind,
    a.Primary_Flag,
    r.Customer_Key,
    b.Account_Nbr,
    r.Site_Id,
    b.Customer_Status,
    b.Inception_Date,
    a.Registration_date,
    CAST(NULL AS varchar(255)) AS User_Permission,
    CAST(NULL AS varchar(255)) AS Preferred_Contact_Method,
    CAST(NULL AS varchar(255)) AS Placeholder2,
    t.TSV_Enrolled_Status,
    mfa.TSV_EMAIL_Flag,
    mfa.TSV_CALL_Flag,
    mfa.TSV_SMS_Flag,
    CAST(NULL AS varchar(255)) AS Preference_Placeholder1,
    CAST(NULL AS varchar(255)) AS Preference_Placeholder2,
    CAST(NULL AS varchar(255)) AS Preferences_Last_Used_Date,
    MAX(r.last_logged_in_date_okta) AS Last_logged_in_date_okta,
    MAX(mob.Last_Logged_In_OS_Cox_App) AS Last_Logged_In_OS_Cox_App,
    CAST(NULL AS varchar(255)) AS Last_Password_Change_Date,
    r.time_key
FROM revenue_data r 
LEFT JOIN customer_dim b ON r.customer_key = b.customer_key
LEFT JOIN guid_data a ON r.customer_key = a.customer_key
LEFT JOIN tsv_data t ON r.customer_key = t.customer_key
LEFT JOIN mfa_data mfa ON a.user_id = mfa.username
LEFT JOIN mob_data mob ON r.customer_key = CAST(mob.customer_key AS double)
GROUP BY a.User_GUID,
    b.Res_Com_Ind,
    a.Primary_Flag,
    r.Customer_Key,
    b.Account_Nbr,
    r.Site_Id,
    b.Customer_Status,
    b.Inception_Date,
    a.Registration_date,
    t.TSV_Enrolled_Status,
    mfa.TSV_EMAIL_Flag,
    mfa.TSV_CALL_Flag,
    mfa.TSV_SMS_Flag,
    r.time_key
"""

In [3]:
profile_sum_dim_new_1_df = spark.sql(profile_sum_dim_new_1)
profile_sum_dim_new_1_df.show()

+---------+-----------+------------+------------+------------+-------+---------------+--------------------+-----------------+---------------+------------------------+------------+-------------------+--------------+-------------+------------+-----------------------+-----------------------+--------------------------+------------------------+-------------------------+-------------------------+----------+
|User_GUID|Res_Com_Ind|Primary_Flag|Customer_Key| Account_Nbr|Site_Id|Customer_Status|      Inception_Date|Registration_date|User_Permission|Preferred_Contact_Method|Placeholder2|TSV_Enrolled_Status|TSV_EMAIL_Flag|TSV_CALL_Flag|TSV_SMS_Flag|Preference_Placeholder1|Preference_Placeholder2|Preferences_Last_Used_Date|Last_logged_in_date_okta|Last_Logged_In_OS_Cox_App|Last_Password_Change_Date|  time_key|
+---------+-----------+------------+------------+------------+-------+---------------+--------------------+-----------------+---------------+------------------------+------------+-----------

In [11]:
profile_sum_dim_new_1_df.count()

Py4JJavaError: An error occurred while calling o87.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 127.0 failed 4 times, most recent failure: Lost task 0.3 in stage 127.0 (TID 5885) (172.36.230.167 executor 8): org.apache.spark.sql.execution.datasources.FileDownloadException: Failed to download file path: s3://cci-edo-data-curated-prod/edw/edw/customer_dim/part-00002-34a1a3f1-092c-41de-99bb-8bc1d27d37f0-c000.snappy.parquet, range: 0-134217728, partition values: [empty row], isDataPresent: false, eTag: 6f2e7dd66d4a61c93d4786b7b1fbdf4e-2
	at org.apache.spark.sql.execution.datasources.AsyncFileDownloader.next(AsyncFileDownloader.scala:142)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.getNextFile(FileScanRDD.scala:423)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:337)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:227)
	at org.

In [ ]:
profile_sum_dim_new_1_df.persist(StorageLevel.MEMORY_AND_DISK)

In [10]:
profile_sum_dim_new_1_df.printSchema()

root
 |-- User_GUID: string (nullable = true)
 |-- Res_Com_Ind: string (nullable = true)
 |-- Primary_Flag: string (nullable = true)
 |-- Customer_Key: double (nullable = true)
 |-- Account_Nbr: double (nullable = true)
 |-- Site_Id: short (nullable = true)
 |-- Customer_Status: string (nullable = true)
 |-- Inception_Date: string (nullable = true)
 |-- Registration_date: timestamp (nullable = true)
 |-- User_Permission: string (nullable = true)
 |-- Preferred_Contact_Method: string (nullable = true)
 |-- Placeholder2: string (nullable = true)
 |-- TSV_Enrolled_Status: string (nullable = true)
 |-- TSV_EMAIL_Flag: integer (nullable = true)
 |-- TSV_CALL_Flag: integer (nullable = true)
 |-- TSV_SMS_Flag: integer (nullable = true)
 |-- Preference_Placeholder1: string (nullable = true)
 |-- Preference_Placeholder2: string (nullable = true)
 |-- Preferences_Last_Used_Date: string (nullable = true)
 |-- Last_logged_in_date_okta: string (nullable = true)
 |-- Last_Logged_In_OS_Cox_App: strin

In [ ]:
profile_sum_dim_new_1_df.count()

In [7]:
profile_sum_dim_new = """
WITH customer_dim AS (
    SELECT 
        b.customer_key AS Customer_Key,
        b.account_nbr AS Account_Nbr,
        b.site_id AS Site_Id,
        b.customer_status_cd AS Customer_Status,
        b.inception_dt AS Inception_Date,
        b.res_comm_ind AS Res_Com_Ind,
        b.test_account_key 
    FROM
        edw.customer_dim b
    WHERE test_account_key=2
),
revenue_data AS (
    SELECT r.customer_key,
           r.site_id,
           r.time_key
    FROM edw.customer_revenue_fact r
    WHERE r.bill_type_key != 2 AND
    to_date(r.time_key, 'yyyy-MM-dd') >= add_months(CURRENT_DATE, -13)
),
guid_data AS (
    SELECT 
        a.customer_key,
        a.create_dt AS Registration_date,
        a.user_id,
        a.prim_customer_flag AS Primary_Flag,
        a.household_member_guid AS User_GUID,
        a.cox_email_address,
        e.email_account_count as Email_Account_Count
    FROM
        edw.customer_guid_dtl_dim a
    JOIN 
    (SELECT cox_email_address, COUNT(DISTINCT customer_key) AS email_account_count
     FROM edw.customer_guid_dtl_dim
     GROUP BY cox_email_address) e ON a.cox_email_address = e.cox_email_address
),
tsv_guid_data as (SELECT cast(t.`household member guid` as varchar(255)) AS tsv_guid FROM ciam_datamodel.tsv_guid_data t),
latest_event_dates AS (
    SELECT
        a.user_id,
        d.outcome_reason,
        MAX(d.event_date) AS latest_event_date
    FROM
        edw.customer_guid_dtl_dim a
        LEFT JOIN ciam.mfa_factors_enrolled d ON a.user_id = d.username
    WHERE
        d.outcome_result = 'SUCCESS'
    GROUP BY
        a.user_id, d.outcome_reason
),
mfa_data AS (
    SELECT DISTINCT
        a.user_id,
        DATE_FORMAT(CAST(o.event_date as DATE), 'yyyy-MM') AS Contact_Month,
        MAX(o.event_date) AS last_logged_in_date_okta,
        MAX(CASE WHEN d.outcome_reason LIKE '%EMAIL%' AND d.event_date = e.latest_event_date THEN 1 ELSE 0 END) AS TSV_EMAIL_Flag,
        MAX(CASE WHEN d.outcome_reason LIKE '%CALL%' AND d.event_date = e.latest_event_date THEN 1 ELSE 0 END) AS TSV_CALL_Flag,
        MAX(CASE WHEN d.outcome_reason LIKE '%SMS%' AND d.event_date = e.latest_event_date THEN 1 ELSE 0 END) AS TSV_SMS_Flag
    FROM
        edw.customer_guid_dtl_dim a
        LEFT JOIN ciam.mfa_factors_enrolled d ON a.user_id = d.username
        LEFT JOIN latest_event_dates e ON a.user_id = e.user_id AND d.outcome_reason = e.outcome_reason
        LEFT JOIN ciam.successful_authentications_okta o ON a.user_id = o.actor_alternateid
    GROUP BY
        a.user_id, DATE_FORMAT(CAST(o.event_date as DATE), 'yyyy-MM')
),
account_details AS (
    SELECT ac.account_nbr,
           ac.vndr_cust_account_key,
           ac.do_not_email AS Email_Opt_Out,
           ac.do_not_call AS Phone_Opt_Out
    FROM camp_mgmt.accounts ac
    LEFT JOIN camp_mgmt.email_addresses em ON em.vndr_cust_account_key = ac.vndr_cust_account_key
),
notification_flags AS (
    SELECT
    cmcnbr AS customer_number,
    MAX(CASE WHEN cmmeth = 'EMAIL' THEN 'Y' ELSE 'N' END) AS Email_Flag,
    MAX(CASE WHEN cmmeth = 'PHONE' THEN 'Y' ELSE 'N' END) AS Phone_Flag,
    MAX(CASE WHEN cmmeth = 'EMAIL' AND cmcffl = 'Y' THEN 'Y' ELSE 'N' END) AS Email_Verified_Flag,
    MAX(CASE WHEN cmmeth = 'PHONE' AND cmcffl = 'Y' THEN 'Y' ELSE 'N' END) AS Phone_Verified_Flag,
    MAX(CASE 
        WHEN cmmeth = 'EMAIL' THEN 
            CASE 
                WHEN cmcfdt IS NOT NULL AND cmcfdt != 0 THEN 
                    CONCAT('20', 
                        SUBSTRING(CAST(cmcfdt AS STRING), 2, 2), '-', 
                        SUBSTRING(CAST(cmcfdt AS STRING), 4, 2), '-', 
                        SUBSTRING(CAST(cmcfdt AS STRING), 6, 2)
                    )
                ELSE NULL 
            END
    END) AS email_verified_date,
    MAX(CASE 
        WHEN cmmeth = 'PHONE' THEN 
            CASE 
                WHEN cmcfdt IS NOT NULL AND cmcfdt != 0 THEN 
                    CONCAT('20', 
                        SUBSTRING(CAST(cmcfdt AS STRING), 2, 2), '-', 
                        SUBSTRING(CAST(cmcfdt AS STRING), 4, 2), '-', 
                        SUBSTRING(CAST(cmcfdt AS STRING), 6, 2)
                    )
                ELSE NULL 
            END
    END) AS phone_verified_date
FROM pstage.stg_all_cust_notification_methods n 
GROUP BY cmcnbr
),
web_data AS (
    SELECT distinct
        d.customer_key,
        DATE_FORMAT(CAST(d.dt AS DATE), 'yyyy-MM') AS Contact_Month,
        MAX(CAST(d.dt AS DATE)) AS Last_Logged_In_Date_Cox_com
    FROM webanalytics.web_contact_history d
    GROUP BY d.customer_key, DATE_FORMAT(CAST(d.dt AS DATE), 'yyyy-MM')
),
mob_data AS (
    SELECT distinct
        mob.customer_key,
        MAX(mob.post_mobileappid) AS Last_Logged_In_App_ID,
        DATE_FORMAT(CAST(mob.dt AS DATE), 'yyyy-MM') AS Contact_Month,
        MAX(CAST(mob.dt AS DATE)) AS Last_Logged_In_Date_Cox_App
    FROM mobile_data_temp.app_contact_history mob
    GROUP BY mob.customer_key, DATE_FORMAT(CAST(mob.dt AS DATE), 'yyyy-MM')
)
SELECT DISTINCT a.User_GUID,
    b.Res_Com_Ind,
    a.Primary_Flag,
    r.Customer_Key,
    b.Account_Nbr,
    r.Site_Id,
    b.Customer_Status,
    b.Inception_Date,
    a.Registration_date,
    CAST(NULL AS varchar(255)) AS User_Permission,
    a.Email_Account_Count,
    nf.Email_Flag,
    nf.Phone_Flag,
    nf.Email_Verified_Flag,
    nf.Phone_Verified_Flag,
    nf.Email_Verified_Date,
    nf.Phone_Verified_Date,
    ad.Email_Opt_Out, 
    ad.Phone_Opt_Out,   
    CAST(NULL AS varchar(255)) AS Preferred_Contact_Method,
    CAST(NULL AS varchar(255)) AS Placeholder2,
    CASE WHEN (t.tsv_guid) IS NOT NULL THEN 'group.user_membership.add'
        ELSE NULL
        END AS TSV_Enrolled_Status,
    mfa.TSV_EMAIL_Flag,
    mfa.TSV_CALL_Flag,
    mfa.TSV_SMS_Flag,
    CAST(NULL AS varchar(255)) AS Preference_Placeholder1,
    CAST(NULL AS varchar(255)) AS Preference_Placeholder2,
    CAST(NULL AS varchar(255)) AS Preferences_Last_Used_Date,
    MAX(mfa.last_logged_in_date_okta) AS last_logged_in_date_okta,
    MAX(w.Last_Logged_In_Date_Cox_com) AS Last_Logged_In_Date_Cox_com,
    MAX(mob.Last_Logged_In_Date_Cox_App) AS Last_Logged_In_Date_Cox_App,
    CASE 
        WHEN mob.Last_Logged_In_App_ID LIKE 'CoxAccount%' THEN 'iOS'
        WHEN mob.Last_Logged_In_App_ID LIKE 'Cox %' THEN 'Android'
        ELSE 'Null'
    END AS Last_Logged_In_OS_Cox_App,
    CAST(NULL AS varchar(255)) AS Last_Password_Change_Date,
    r.time_key
FROM revenue_data r 
LEFT JOIN customer_dim b ON r.customer_key = b.customer_key
LEFT JOIN guid_data a ON r.customer_key = a.customer_key
INNER JOIN tsv_guid_data t ON a.User_GUID = t.tsv_guid
LEFT JOIN web_data w
    ON r.customer_key = CAST(w.customer_key AS double) AND w.Contact_Month <= r.time_key
LEFT JOIN mob_data mob
    ON r.customer_key = CAST(mob.customer_key AS double) AND mob.Contact_Month <= r.time_key
LEFT JOIN mfa_data mfa ON a.user_id = mfa.user_id AND mfa.Contact_Month <= r.time_key
LEFT JOIN account_details ad ON b.account_nbr = ad.account_nbr
LEFT JOIN notification_flags nf ON b.account_nbr = nf.customer_number
GROUP BY
    a.User_GUID,
    b.Res_Com_Ind,
    a.Primary_Flag,
    r.Customer_Key,
    b.Account_Nbr,
    r.Site_Id,
    b.Customer_Status,
    b.Inception_Date,
    a.Registration_date,
    a.Email_Account_Count,
    nf.Email_Flag,
    nf.Phone_Flag,
    nf.Email_Verified_Flag,
    nf.Phone_Verified_Flag,
    nf.Email_Verified_Date,
    nf.Phone_Verified_Date,
    ad.Email_Opt_Out, 
    ad.Phone_Opt_Out,   
    CASE WHEN (t.tsv_guid) IS NOT NULL THEN 'group.user_membership.add'
        ELSE NULL
        END,
    mfa.TSV_EMAIL_Flag,
    mfa.TSV_CALL_Flag,
    mfa.TSV_SMS_Flag,
    CASE 
        WHEN mob.Last_Logged_In_App_ID LIKE 'CoxAccount%' THEN 'iOS'
        WHEN mob.Last_Logged_In_App_ID LIKE 'Cox %' THEN 'Android'
        ELSE 'Null'
    END,
    r.time_key
"""

In [8]:
profile_sum_dim_new_df = spark.sql(profile_sum_dim_new)
profile_sum_dim_new_df.show()

+--------------------+-----------+------------+------------+------------+-------+---------------+--------------------+-------------------+---------------+-------------------+----------+----------+-------------------+-------------------+-------------------+-------------------+-------------+-------------+------------------------+------------+--------------------+--------------+-------------+------------+-----------------------+-----------------------+--------------------------+------------------------+---------------------------+---------------------------+-------------------------+-------------------------+----------+
|           User_GUID|Res_Com_Ind|Primary_Flag|Customer_Key| Account_Nbr|Site_Id|Customer_Status|      Inception_Date|  Registration_date|User_Permission|Email_Account_Count|Email_Flag|Phone_Flag|Email_Verified_Flag|Phone_Verified_Flag|Email_Verified_Date|Phone_Verified_Date|Email_Opt_Out|Phone_Opt_Out|Preferred_Contact_Method|Placeholder2| TSV_Enrolled_Status|TSV_EMAIL_Fl

In [3]:
profile_sum_dim_new_df.count()

NameError: name 'profile_sum_dim_new_df' is not defined


In [9]:
profile_sum_dim_new_df.persist(StorageLevel.MEMORY_AND_DISK)

DataFrame[User_GUID: string, Res_Com_Ind: string, Primary_Flag: string, Customer_Key: double, Account_Nbr: double, Site_Id: smallint, Customer_Status: string, Inception_Date: string, Registration_date: timestamp, User_Permission: string, Email_Account_Count: bigint, Email_Flag: string, Phone_Flag: string, Email_Verified_Flag: string, Phone_Verified_Flag: string, Email_Verified_Date: string, Phone_Verified_Date: string, Email_Opt_Out: string, Phone_Opt_Out: string, Preferred_Contact_Method: string, Placeholder2: string, TSV_Enrolled_Status: string, TSV_EMAIL_Flag: int, TSV_CALL_Flag: int, TSV_SMS_Flag: int, Preference_Placeholder1: string, Preference_Placeholder2: string, Preferences_Last_Used_Date: string, last_logged_in_date_okta: string, Last_Logged_In_Date_Cox_com: date, Last_Logged_In_Date_Cox_App: date, Last_Logged_In_OS_Cox_App: string, Last_Password_Change_Date: string, time_key: string]


In [10]:
profile_sum_dim_new_df.printSchema()

root
 |-- User_GUID: string (nullable = true)
 |-- Res_Com_Ind: string (nullable = true)
 |-- Primary_Flag: string (nullable = true)
 |-- Customer_Key: double (nullable = true)
 |-- Account_Nbr: double (nullable = true)
 |-- Site_Id: short (nullable = true)
 |-- Customer_Status: string (nullable = true)
 |-- Inception_Date: string (nullable = true)
 |-- Registration_date: timestamp (nullable = true)
 |-- User_Permission: string (nullable = true)
 |-- Email_Account_Count: long (nullable = true)
 |-- Email_Flag: string (nullable = true)
 |-- Phone_Flag: string (nullable = true)
 |-- Email_Verified_Flag: string (nullable = true)
 |-- Phone_Verified_Flag: string (nullable = true)
 |-- Email_Verified_Date: string (nullable = true)
 |-- Phone_Verified_Date: string (nullable = true)
 |-- Email_Opt_Out: string (nullable = true)
 |-- Phone_Opt_Out: string (nullable = true)
 |-- Preferred_Contact_Method: string (nullable = true)
 |-- Placeholder2: string (nullable = true)
 |-- TSV_Enrolled_Statu

In [ ]:
profile_sum_dim_new_df.count()